In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
tf.enable_eager_execution()

## data load

In [23]:
def load_mnist():
    (train_data, train_labels),(test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data,axis=-1)  # 위 코드는 [n,28,28]이므로 맨뒤(-1)에 채널을 추가한다. 
                                                  #tensorflow 가 input으로 받을 때 [batch_size,height,width,channel]로 받아야한다
    test_data = np.expand_dims(test_data,axis=-1)
    
    train_data, test_data = normalize(train_data, test_data)
    
    train_labels = to_categorical(train_labels, 10) #은 label의 갯수. -> onehot encoding
    test_labels = to_categorical(test_labels,10)    # ex) 7 을 0000000100
    
    return train_data,train_labels, test_data, test_labels
    
def normalize(train_data,test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    
    return train_data, test_data

## create network

In [18]:
def flatten():
    return tf.keras.layers.Flatten()    #[N,28,28,1] -> [N,784]

def dense(channel, weight_init):
    return tf.keras.layers.Dense(units = channel,  # output으로 나가는 채널의 갯수
                                use_bias = True,   # bias 사용 유무
                                kernel_initializer = weight_init)  # weight 시작값

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [19]:
# 모델 만들기
def create_model(label_dim):
    weight_init = tf.keras.initializers.RandomNormal()  # weight 초기화방법(=randomnormal). 아~래에서 xivier로 가중치 초기화를 잘 해보자
    
    model = tf.keras.Sequential()  # 층층히 쌓아가기 위한 리스트 자료구조 타입
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256,weight_init))
        model.add(relu())
        
    model.add(dense(label_dim,weight_init))
    
    return model

## Define Loss

In [59]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)  ## True-> 아~래의 dropout을 사용한다는 뜻. loss 는 train이겠고
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)  ## False-> 아~래의 dropout을 사용하지 않는다는 뜻.  accuracy는 test겠지
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels,-1))  # logit과 label 의 shape = [batchsize,label_dim]
    accuracy =tf.reduce_mean(tf.cast(prediction, tf.float32))  # prediction이 boolean이기 때문에 계산에 용이하게 숫자로 바꿔주자

    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(model,images,labels)
    
    return tape.gradient(loss_value, model.variables)

## Experiments(parameters)

In [24]:
# dataset
train_x, train_y, test_x, test_y = load_mnist()

In [61]:
# parameters
learning_rate = 0.001
batch_size=128
training_epochs=1
training_iterations = len(train_x)//batch_size

start_epoch = 0
start_iteration = 0

label_dim=10

In [36]:
# Graph Input usin Dataset API
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_y)).\
shuffle(buffer_size=100000).\
prefetch(buffer_size=batch_size).\
batch(batch_size).\
repeat()

# shuffle(buffer_size=100000) : batch로 쪼개서 섞어 보내기 위함. train과 test size보다 큰 숫자로 지정하자.  train 6만, test 10만
# prefetch(buffer_size=batch_size) : 학습도중 다음 데이터를 미리 메모리에 올려 놓아라 (속도 향상)
# batch(batch_size) : 한번에 던져줄 batch 

In [37]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_x,test_y)).\
shuffle(buffer_size=100000).\
prefetch(buffer_size=batch_size).\
batch(batch_size).\
repeat()


## Experiments(model)

In [53]:
# Dataset Iterator
train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

In [56]:
# Model
network = create_model(label_dim)

In [57]:
# training
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [58]:
checkpoint = tf.train.Checkpoint(dnn=network) # 오류로 멈췄을 때, 이전에 학습되어 저장되어져 있는 데이터를 불러오기 위함
global_step = tf.train.create_global_step() #각각의 weight가 몇번의 iteration인지

In [65]:
counter = 0
for epoch in range(start_epoch, training_epochs):
    for idx in range(start_iteration, training_iterations):
        train_input, train_label = train_iterator.get_next()   # train_iterator에서 각각의 인자를 받아옴
        
        grads = grad(network, train_input, train_label)
        optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables),
                                 global_step=global_step)
        
        train_loss = loss_fn(network, train_input, train_label)
        train_accuracy = accuracy_fn(network, train_input, train_label)
        
        
        
        test_input, test_label = test_iterator.get_next()
        test_accuracy = accuracy_fn(network, test_input, test_label)
        
        print("Epoch: [%2d] [%5d/%5d], train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, train_loss, train_accuracy,test_accuracy))
        
        counter+=1



Epoch: [ 0] [    0/  468], train_loss: 0.12298617, train_accuracy: 0.9609, test_Accuracy: 0.9531
Epoch: [ 0] [    1/  468], train_loss: 0.08924934, train_accuracy: 0.9844, test_Accuracy: 0.9453
Epoch: [ 0] [    2/  468], train_loss: 0.10160820, train_accuracy: 0.9766, test_Accuracy: 0.9688
Epoch: [ 0] [    3/  468], train_loss: 0.05426317, train_accuracy: 1.0000, test_Accuracy: 0.9531
Epoch: [ 0] [    4/  468], train_loss: 0.12819479, train_accuracy: 0.9375, test_Accuracy: 0.8750
Epoch: [ 0] [    5/  468], train_loss: 0.17527053, train_accuracy: 0.9375, test_Accuracy: 0.9531
Epoch: [ 0] [    6/  468], train_loss: 0.08772154, train_accuracy: 0.9609, test_Accuracy: 0.9844
Epoch: [ 0] [    7/  468], train_loss: 0.13646305, train_accuracy: 0.9688, test_Accuracy: 0.9531
Epoch: [ 0] [    8/  468], train_loss: 0.13224588, train_accuracy: 0.9688, test_Accuracy: 0.9375
Epoch: [ 0] [    9/  468], train_loss: 0.06550828, train_accuracy: 0.9844, test_Accuracy: 0.9453
Epoch: [ 0] [   10/  468], tra

Epoch: [ 0] [   87/  468], train_loss: 0.07917389, train_accuracy: 0.9922, test_Accuracy: 0.9531
Epoch: [ 0] [   88/  468], train_loss: 0.03270952, train_accuracy: 1.0000, test_Accuracy: 0.9609
Epoch: [ 0] [   89/  468], train_loss: 0.16024330, train_accuracy: 0.9609, test_Accuracy: 0.9922
Epoch: [ 0] [   90/  468], train_loss: 0.09946445, train_accuracy: 0.9609, test_Accuracy: 0.9453
Epoch: [ 0] [   91/  468], train_loss: 0.08834680, train_accuracy: 0.9609, test_Accuracy: 0.9297
Epoch: [ 0] [   92/  468], train_loss: 0.11981144, train_accuracy: 0.9531, test_Accuracy: 0.9844
Epoch: [ 0] [   93/  468], train_loss: 0.05471756, train_accuracy: 0.9922, test_Accuracy: 0.9688
Epoch: [ 0] [   94/  468], train_loss: 0.16909006, train_accuracy: 0.9453, test_Accuracy: 0.9688
Epoch: [ 0] [   95/  468], train_loss: 0.06851117, train_accuracy: 0.9844, test_Accuracy: 0.9453
Epoch: [ 0] [   96/  468], train_loss: 0.17851573, train_accuracy: 0.9453, test_Accuracy: 0.9453
Epoch: [ 0] [   97/  468], tra

Epoch: [ 0] [  172/  468], train_loss: 0.09122168, train_accuracy: 0.9844, test_Accuracy: 0.9766
Epoch: [ 0] [  173/  468], train_loss: 0.05398107, train_accuracy: 0.9844, test_Accuracy: 0.9297
Epoch: [ 0] [  174/  468], train_loss: 0.08852313, train_accuracy: 0.9766, test_Accuracy: 0.9453
Epoch: [ 0] [  175/  468], train_loss: 0.12222172, train_accuracy: 0.9688, test_Accuracy: 0.9688
Epoch: [ 0] [  176/  468], train_loss: 0.16835482, train_accuracy: 0.9453, test_Accuracy: 0.9688
Epoch: [ 0] [  177/  468], train_loss: 0.04249917, train_accuracy: 0.9922, test_Accuracy: 0.9609
Epoch: [ 0] [  178/  468], train_loss: 0.04263305, train_accuracy: 0.9844, test_Accuracy: 0.9766
Epoch: [ 0] [  179/  468], train_loss: 0.14681886, train_accuracy: 0.9609, test_Accuracy: 0.9766
Epoch: [ 0] [  180/  468], train_loss: 0.11857212, train_accuracy: 0.9688, test_Accuracy: 0.9766
Epoch: [ 0] [  181/  468], train_loss: 0.05811702, train_accuracy: 0.9844, test_Accuracy: 0.9609
Epoch: [ 0] [  182/  468], tra

Epoch: [ 0] [  258/  468], train_loss: 0.07053612, train_accuracy: 0.9688, test_Accuracy: 0.9609
Epoch: [ 0] [  259/  468], train_loss: 0.06220806, train_accuracy: 0.9766, test_Accuracy: 0.9844
Epoch: [ 0] [  260/  468], train_loss: 0.08871151, train_accuracy: 0.9766, test_Accuracy: 0.9375
Epoch: [ 0] [  261/  468], train_loss: 0.04955969, train_accuracy: 0.9844, test_Accuracy: 0.9609
Epoch: [ 0] [  262/  468], train_loss: 0.07128318, train_accuracy: 0.9766, test_Accuracy: 0.9922
Epoch: [ 0] [  263/  468], train_loss: 0.05927230, train_accuracy: 0.9844, test_Accuracy: 0.9453
Epoch: [ 0] [  264/  468], train_loss: 0.04332165, train_accuracy: 0.9922, test_Accuracy: 0.9766
Epoch: [ 0] [  265/  468], train_loss: 0.17498711, train_accuracy: 0.9688, test_Accuracy: 0.9688
Epoch: [ 0] [  266/  468], train_loss: 0.08330162, train_accuracy: 0.9688, test_Accuracy: 0.9297
Epoch: [ 0] [  267/  468], train_loss: 0.13390630, train_accuracy: 0.9531, test_Accuracy: 0.9375
Epoch: [ 0] [  268/  468], tra

Epoch: [ 0] [  343/  468], train_loss: 0.08778682, train_accuracy: 0.9844, test_Accuracy: 0.9609
Epoch: [ 0] [  344/  468], train_loss: 0.08432184, train_accuracy: 0.9922, test_Accuracy: 0.9531
Epoch: [ 0] [  345/  468], train_loss: 0.14748415, train_accuracy: 0.9609, test_Accuracy: 0.9453
Epoch: [ 0] [  346/  468], train_loss: 0.06902763, train_accuracy: 0.9688, test_Accuracy: 0.9453
Epoch: [ 0] [  347/  468], train_loss: 0.03597110, train_accuracy: 0.9922, test_Accuracy: 0.9766
Epoch: [ 0] [  348/  468], train_loss: 0.12450439, train_accuracy: 0.9766, test_Accuracy: 0.9766
Epoch: [ 0] [  349/  468], train_loss: 0.08937211, train_accuracy: 0.9766, test_Accuracy: 0.9766
Epoch: [ 0] [  350/  468], train_loss: 0.11959171, train_accuracy: 0.9375, test_Accuracy: 0.9844
Epoch: [ 0] [  351/  468], train_loss: 0.05384803, train_accuracy: 0.9688, test_Accuracy: 0.9688
Epoch: [ 0] [  352/  468], train_loss: 0.03527414, train_accuracy: 0.9922, test_Accuracy: 0.9688
Epoch: [ 0] [  353/  468], tra

Epoch: [ 0] [  429/  468], train_loss: 0.08357729, train_accuracy: 0.9844, test_Accuracy: 0.9375
Epoch: [ 0] [  430/  468], train_loss: 0.04068755, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [  431/  468], train_loss: 0.10685484, train_accuracy: 0.9766, test_Accuracy: 0.9297
Epoch: [ 0] [  432/  468], train_loss: 0.07263946, train_accuracy: 0.9688, test_Accuracy: 0.9375
Epoch: [ 0] [  433/  468], train_loss: 0.05505894, train_accuracy: 0.9844, test_Accuracy: 0.9766
Epoch: [ 0] [  434/  468], train_loss: 0.05290578, train_accuracy: 0.9766, test_Accuracy: 0.9688
Epoch: [ 0] [  435/  468], train_loss: 0.04164994, train_accuracy: 0.9844, test_Accuracy: 0.9531
Epoch: [ 0] [  436/  468], train_loss: 0.07117843, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [  437/  468], train_loss: 0.09240225, train_accuracy: 0.9766, test_Accuracy: 0.9609
Epoch: [ 0] [  438/  468], train_loss: 0.09052532, train_accuracy: 0.9766, test_Accuracy: 0.9609
Epoch: [ 0] [  439/  468], tra

In [66]:
# 모델 만들기
def create_model(label_dim):
    weight_init = tf.keras.initializers.glorot_uniform()  # weight 초기화방법(=randomnormal, glorot_uniform(),he_uniform)()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256,weight_init))
        model.add(relu())
        
    model.add(dense(label_dim,weight_init))
    
    return model

In [67]:
counter = 0
for epoch in range(start_epoch, training_epochs):
    for idx in range(start_iteration, training_iterations):
        train_input, train_label = train_iterator.get_next()   # train_iterator에서 각각의 인자를 받아옴
        
        grads = grad(network, train_input, train_label)
        optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables),
                                 global_step=global_step)
        
        train_loss = loss_fn(network, train_input, train_label)
        train_accuracy = accuracy_fn(network, train_input, train_label)
        
        
        
        test_input, test_label = test_iterator.get_next()
        test_accuracy = accuracy_fn(network, test_input, test_label)
        
        print("Epoch: [%2d] [%5d/%5d], train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, train_loss, train_accuracy,test_accuracy))
        
        counter+=1

Epoch: [ 0] [    0/  468], train_loss: 0.11536219, train_accuracy: 0.9922, test_Accuracy: 0.9609
Epoch: [ 0] [    1/  468], train_loss: 0.04324286, train_accuracy: 1.0000, test_Accuracy: 0.9688
Epoch: [ 0] [    2/  468], train_loss: 0.05443031, train_accuracy: 0.9844, test_Accuracy: 0.9844
Epoch: [ 0] [    3/  468], train_loss: 0.06196278, train_accuracy: 0.9844, test_Accuracy: 0.9531
Epoch: [ 0] [    4/  468], train_loss: 0.04889655, train_accuracy: 0.9844, test_Accuracy: 0.9766
Epoch: [ 0] [    5/  468], train_loss: 0.01454109, train_accuracy: 1.0000, test_Accuracy: 0.9766
Epoch: [ 0] [    6/  468], train_loss: 0.04021512, train_accuracy: 0.9922, test_Accuracy: 0.9688
Epoch: [ 0] [    7/  468], train_loss: 0.07646070, train_accuracy: 0.9766, test_Accuracy: 0.9531
Epoch: [ 0] [    8/  468], train_loss: 0.02691969, train_accuracy: 0.9922, test_Accuracy: 0.9688
Epoch: [ 0] [    9/  468], train_loss: 0.07670771, train_accuracy: 0.9844, test_Accuracy: 0.9531
Epoch: [ 0] [   10/  468], tra

Epoch: [ 0] [   86/  468], train_loss: 0.06298696, train_accuracy: 0.9688, test_Accuracy: 0.9844
Epoch: [ 0] [   87/  468], train_loss: 0.03358605, train_accuracy: 0.9766, test_Accuracy: 0.9844
Epoch: [ 0] [   88/  468], train_loss: 0.05092092, train_accuracy: 0.9922, test_Accuracy: 1.0000
Epoch: [ 0] [   89/  468], train_loss: 0.03212567, train_accuracy: 0.9922, test_Accuracy: 0.9375
Epoch: [ 0] [   90/  468], train_loss: 0.06515371, train_accuracy: 0.9844, test_Accuracy: 0.9609
Epoch: [ 0] [   91/  468], train_loss: 0.04833888, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [   92/  468], train_loss: 0.03232938, train_accuracy: 0.9922, test_Accuracy: 0.9609
Epoch: [ 0] [   93/  468], train_loss: 0.04622339, train_accuracy: 0.9844, test_Accuracy: 0.9609
Epoch: [ 0] [   94/  468], train_loss: 0.04534978, train_accuracy: 0.9766, test_Accuracy: 0.9766
Epoch: [ 0] [   95/  468], train_loss: 0.02876530, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [   96/  468], tra

Epoch: [ 0] [  171/  468], train_loss: 0.08378755, train_accuracy: 0.9844, test_Accuracy: 0.9844
Epoch: [ 0] [  172/  468], train_loss: 0.02927535, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [  173/  468], train_loss: 0.05490058, train_accuracy: 0.9844, test_Accuracy: 0.9609
Epoch: [ 0] [  174/  468], train_loss: 0.06069535, train_accuracy: 0.9766, test_Accuracy: 0.9453
Epoch: [ 0] [  175/  468], train_loss: 0.04876019, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [  176/  468], train_loss: 0.05426048, train_accuracy: 0.9922, test_Accuracy: 0.9609
Epoch: [ 0] [  177/  468], train_loss: 0.14963445, train_accuracy: 0.9531, test_Accuracy: 0.9766
Epoch: [ 0] [  178/  468], train_loss: 0.03956731, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [  179/  468], train_loss: 0.07249179, train_accuracy: 0.9688, test_Accuracy: 0.9844
Epoch: [ 0] [  180/  468], train_loss: 0.08652374, train_accuracy: 0.9844, test_Accuracy: 0.9844
Epoch: [ 0] [  181/  468], tra

Epoch: [ 0] [  257/  468], train_loss: 0.07211620, train_accuracy: 0.9766, test_Accuracy: 0.9688
Epoch: [ 0] [  258/  468], train_loss: 0.07307281, train_accuracy: 0.9688, test_Accuracy: 0.9688
Epoch: [ 0] [  259/  468], train_loss: 0.07495389, train_accuracy: 0.9844, test_Accuracy: 0.9766
Epoch: [ 0] [  260/  468], train_loss: 0.03850458, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [  261/  468], train_loss: 0.07129214, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [  262/  468], train_loss: 0.06925414, train_accuracy: 0.9766, test_Accuracy: 0.9922
Epoch: [ 0] [  263/  468], train_loss: 0.11661258, train_accuracy: 0.9766, test_Accuracy: 0.9688
Epoch: [ 0] [  264/  468], train_loss: 0.06882578, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [  265/  468], train_loss: 0.14996201, train_accuracy: 0.9531, test_Accuracy: 0.9688
Epoch: [ 0] [  266/  468], train_loss: 0.07251324, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [  267/  468], tra

Epoch: [ 0] [  343/  468], train_loss: 0.02638493, train_accuracy: 0.9922, test_Accuracy: 0.9922
Epoch: [ 0] [  344/  468], train_loss: 0.04097566, train_accuracy: 0.9688, test_Accuracy: 0.9688
Epoch: [ 0] [  345/  468], train_loss: 0.03974193, train_accuracy: 0.9844, test_Accuracy: 0.9922
Epoch: [ 0] [  346/  468], train_loss: 0.04166885, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [  347/  468], train_loss: 0.05036782, train_accuracy: 0.9844, test_Accuracy: 0.9844
Epoch: [ 0] [  348/  468], train_loss: 0.04291295, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [  349/  468], train_loss: 0.08475538, train_accuracy: 0.9766, test_Accuracy: 0.9609
Epoch: [ 0] [  350/  468], train_loss: 0.04303076, train_accuracy: 0.9922, test_Accuracy: 0.9609
Epoch: [ 0] [  351/  468], train_loss: 0.02222008, train_accuracy: 0.9922, test_Accuracy: 0.9766
Epoch: [ 0] [  352/  468], train_loss: 0.01855426, train_accuracy: 1.0000, test_Accuracy: 0.9609
Epoch: [ 0] [  353/  468], tra

Epoch: [ 0] [  429/  468], train_loss: 0.03157137, train_accuracy: 0.9922, test_Accuracy: 0.9922
Epoch: [ 0] [  430/  468], train_loss: 0.03056176, train_accuracy: 0.9922, test_Accuracy: 0.9688
Epoch: [ 0] [  431/  468], train_loss: 0.02913946, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [  432/  468], train_loss: 0.06053023, train_accuracy: 0.9844, test_Accuracy: 0.9375
Epoch: [ 0] [  433/  468], train_loss: 0.02497204, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [  434/  468], train_loss: 0.05417791, train_accuracy: 0.9922, test_Accuracy: 0.9609
Epoch: [ 0] [  435/  468], train_loss: 0.04009063, train_accuracy: 0.9844, test_Accuracy: 0.9766
Epoch: [ 0] [  436/  468], train_loss: 0.03262145, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [  437/  468], train_loss: 0.05135334, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [  438/  468], train_loss: 0.04074086, train_accuracy: 0.9922, test_Accuracy: 0.9531
Epoch: [ 0] [  439/  468], tra

weight 초기화 방법을 바꾸니 모델이 향상되었다.

## Drop out


오버 피팅을 줄이기 위함

학습 도중에 일부 노드를 끄고 모델을 만듬

In [71]:
def dropout(rate):
    return tf.keras.layers.Dropout(rate)  # rate: 네트워크/노드를 몇퍼센트 확률로 끌 것이냐 0~1사이

In [72]:
# 모델 만들기
def create_model(label_dim):
    weight_init = tf.keras.initializers.RandomNormal()  # weight 초기화방법(=randomnormal, glorot_uniform(),he_uniform)()
    
    model = tf.keras.Sequential()  # 층층히 쌓아가기 위한 리스트 자료구조 타입
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256,weight_init))
        model.add(relu())
        model.add(dropout(rate=0.5))  ## 추가
        
    model.add(dense(label_dim,weight_init))
    
    return model

In [73]:
counter = 0
for epoch in range(start_epoch, training_epochs):
    for idx in range(start_iteration, training_iterations):
        train_input, train_label = train_iterator.get_next()   # train_iterator에서 각각의 인자를 받아옴
        
        grads = grad(network, train_input, train_label)
        optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables),
                                 global_step=global_step)
        
        train_loss = loss_fn(network, train_input, train_label)
        train_accuracy = accuracy_fn(network, train_input, train_label)
        
        
        
        test_input, test_label = test_iterator.get_next()
        test_accuracy = accuracy_fn(network, test_input, test_label)
        
        print("Epoch: [%2d] [%5d/%5d], train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, train_loss, train_accuracy,test_accuracy))
        
        counter+=1

Epoch: [ 0] [    0/  468], train_loss: 0.01714325, train_accuracy: 1.0000, test_Accuracy: 0.9688
Epoch: [ 0] [    1/  468], train_loss: 0.03913524, train_accuracy: 0.9922, test_Accuracy: 0.9766
Epoch: [ 0] [    2/  468], train_loss: 0.02153061, train_accuracy: 1.0000, test_Accuracy: 0.9766
Epoch: [ 0] [    3/  468], train_loss: 0.01930962, train_accuracy: 0.9922, test_Accuracy: 0.9609
Epoch: [ 0] [    4/  468], train_loss: 0.05546836, train_accuracy: 0.9766, test_Accuracy: 0.9766
Epoch: [ 0] [    5/  468], train_loss: 0.02362581, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [    6/  468], train_loss: 0.04541276, train_accuracy: 0.9844, test_Accuracy: 0.9922
Epoch: [ 0] [    7/  468], train_loss: 0.04018174, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [    8/  468], train_loss: 0.02513557, train_accuracy: 1.0000, test_Accuracy: 0.9844
Epoch: [ 0] [    9/  468], train_loss: 0.04981327, train_accuracy: 0.9844, test_Accuracy: 0.9766
Epoch: [ 0] [   10/  468], tra

Epoch: [ 0] [   86/  468], train_loss: 0.01915640, train_accuracy: 1.0000, test_Accuracy: 0.9766
Epoch: [ 0] [   87/  468], train_loss: 0.04631998, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [   88/  468], train_loss: 0.04413083, train_accuracy: 0.9844, test_Accuracy: 1.0000
Epoch: [ 0] [   89/  468], train_loss: 0.03332704, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [   90/  468], train_loss: 0.03720510, train_accuracy: 0.9844, test_Accuracy: 0.9844
Epoch: [ 0] [   91/  468], train_loss: 0.03162800, train_accuracy: 0.9844, test_Accuracy: 0.9766
Epoch: [ 0] [   92/  468], train_loss: 0.03451988, train_accuracy: 0.9922, test_Accuracy: 0.9531
Epoch: [ 0] [   93/  468], train_loss: 0.02770961, train_accuracy: 0.9844, test_Accuracy: 0.9609
Epoch: [ 0] [   94/  468], train_loss: 0.02208605, train_accuracy: 0.9922, test_Accuracy: 0.9531
Epoch: [ 0] [   95/  468], train_loss: 0.03981322, train_accuracy: 0.9844, test_Accuracy: 1.0000
Epoch: [ 0] [   96/  468], tra

Epoch: [ 0] [  172/  468], train_loss: 0.02071447, train_accuracy: 0.9922, test_Accuracy: 0.9766
Epoch: [ 0] [  173/  468], train_loss: 0.01419684, train_accuracy: 1.0000, test_Accuracy: 0.9766
Epoch: [ 0] [  174/  468], train_loss: 0.04351510, train_accuracy: 0.9922, test_Accuracy: 1.0000
Epoch: [ 0] [  175/  468], train_loss: 0.03206525, train_accuracy: 0.9844, test_Accuracy: 0.9844
Epoch: [ 0] [  176/  468], train_loss: 0.03639185, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [  177/  468], train_loss: 0.01999781, train_accuracy: 0.9922, test_Accuracy: 0.9766
Epoch: [ 0] [  178/  468], train_loss: 0.07554627, train_accuracy: 0.9688, test_Accuracy: 0.9844
Epoch: [ 0] [  179/  468], train_loss: 0.05106471, train_accuracy: 0.9766, test_Accuracy: 0.9766
Epoch: [ 0] [  180/  468], train_loss: 0.05083221, train_accuracy: 0.9844, test_Accuracy: 0.9766
Epoch: [ 0] [  181/  468], train_loss: 0.03085119, train_accuracy: 0.9922, test_Accuracy: 0.9922
Epoch: [ 0] [  182/  468], tra

Epoch: [ 0] [  257/  468], train_loss: 0.02962333, train_accuracy: 1.0000, test_Accuracy: 0.9609
Epoch: [ 0] [  258/  468], train_loss: 0.01922390, train_accuracy: 1.0000, test_Accuracy: 0.9844
Epoch: [ 0] [  259/  468], train_loss: 0.02636333, train_accuracy: 0.9922, test_Accuracy: 0.9766
Epoch: [ 0] [  260/  468], train_loss: 0.05143829, train_accuracy: 0.9766, test_Accuracy: 0.9922
Epoch: [ 0] [  261/  468], train_loss: 0.02744152, train_accuracy: 0.9922, test_Accuracy: 0.9531
Epoch: [ 0] [  262/  468], train_loss: 0.02638765, train_accuracy: 1.0000, test_Accuracy: 0.9609
Epoch: [ 0] [  263/  468], train_loss: 0.01356063, train_accuracy: 1.0000, test_Accuracy: 0.9922
Epoch: [ 0] [  264/  468], train_loss: 0.09387445, train_accuracy: 0.9688, test_Accuracy: 0.9844
Epoch: [ 0] [  265/  468], train_loss: 0.03365401, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [  266/  468], train_loss: 0.00678434, train_accuracy: 1.0000, test_Accuracy: 0.9844
Epoch: [ 0] [  267/  468], tra

Epoch: [ 0] [  344/  468], train_loss: 0.05321019, train_accuracy: 0.9766, test_Accuracy: 0.9766
Epoch: [ 0] [  345/  468], train_loss: 0.04337841, train_accuracy: 0.9922, test_Accuracy: 0.9766
Epoch: [ 0] [  346/  468], train_loss: 0.03613521, train_accuracy: 0.9844, test_Accuracy: 0.9844
Epoch: [ 0] [  347/  468], train_loss: 0.00916296, train_accuracy: 1.0000, test_Accuracy: 0.9375
Epoch: [ 0] [  348/  468], train_loss: 0.01979989, train_accuracy: 1.0000, test_Accuracy: 0.9609
Epoch: [ 0] [  349/  468], train_loss: 0.01687472, train_accuracy: 1.0000, test_Accuracy: 0.9922
Epoch: [ 0] [  350/  468], train_loss: 0.02933063, train_accuracy: 0.9922, test_Accuracy: 0.9688
Epoch: [ 0] [  351/  468], train_loss: 0.04125786, train_accuracy: 0.9922, test_Accuracy: 0.9688
Epoch: [ 0] [  352/  468], train_loss: 0.05619052, train_accuracy: 0.9844, test_Accuracy: 0.9766
Epoch: [ 0] [  353/  468], train_loss: 0.03396711, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [  354/  468], tra

Epoch: [ 0] [  430/  468], train_loss: 0.07934204, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [  431/  468], train_loss: 0.01900129, train_accuracy: 1.0000, test_Accuracy: 0.9844
Epoch: [ 0] [  432/  468], train_loss: 0.09619398, train_accuracy: 0.9688, test_Accuracy: 0.9844
Epoch: [ 0] [  433/  468], train_loss: 0.03441369, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [  434/  468], train_loss: 0.02077082, train_accuracy: 0.9922, test_Accuracy: 0.9766
Epoch: [ 0] [  435/  468], train_loss: 0.05390343, train_accuracy: 0.9922, test_Accuracy: 0.9766
Epoch: [ 0] [  436/  468], train_loss: 0.05080147, train_accuracy: 0.9922, test_Accuracy: 0.9922
Epoch: [ 0] [  437/  468], train_loss: 0.04371664, train_accuracy: 0.9844, test_Accuracy: 0.9844
Epoch: [ 0] [  438/  468], train_loss: 0.01495486, train_accuracy: 1.0000, test_Accuracy: 0.9844
Epoch: [ 0] [  439/  468], train_loss: 0.08265525, train_accuracy: 0.9766, test_Accuracy: 0.9844
Epoch: [ 0] [  440/  468], tra

## Batch Normalization

네트워크를 지나가면서 분포가 망가진다 : Internal Covariate Shift

이런 현상을 막는 즉, input data의 분포를 일정하게 유지하기 위함

drop out 필요없으므로 batch_norm 이라는 함수를 하나 만들어 주자

In [74]:
def batch_norm():
    return tf.keras.layers.BatchNormalization()

In [75]:
# 모델 만들기
def create_model(label_dim):
    weight_init = tf.keras.initializers.RandomNormal()  # weight 초기화방법(=randomnormal, glorot_uniform(),he_uniform)()
    
    model = tf.keras.Sequential()  # 층층히 쌓아가기 위한 리스트 자료구조 타입
    model.add(flatten())
    
    for i in range(2):                       ## 순서 A-B-C나 B-C-A
        model.add(dense(256,weight_init))  # A : layer
        model.add(batch_norm())  ## 수정   # B : norm
        model.add(relu())                  # C : activation

        
    model.add(dense(label_dim,weight_init))
    
    return model

In [76]:
counter = 0
for epoch in range(start_epoch, training_epochs):
    for idx in range(start_iteration, training_iterations):
        train_input, train_label = train_iterator.get_next()   # train_iterator에서 각각의 인자를 받아옴
        
        grads = grad(network, train_input, train_label)
        optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables),
                                 global_step=global_step)
        
        train_loss = loss_fn(network, train_input, train_label)
        train_accuracy = accuracy_fn(network, train_input, train_label)
        
        
        
        test_input, test_label = test_iterator.get_next()
        test_accuracy = accuracy_fn(network, test_input, test_label)
        
        print("Epoch: [%2d] [%5d/%5d], train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, train_loss, train_accuracy,test_accuracy))
        
        counter+=1

Epoch: [ 0] [    0/  468], train_loss: 0.01421615, train_accuracy: 1.0000, test_Accuracy: 0.9453
Epoch: [ 0] [    1/  468], train_loss: 0.02609358, train_accuracy: 1.0000, test_Accuracy: 0.9922
Epoch: [ 0] [    2/  468], train_loss: 0.01102579, train_accuracy: 1.0000, test_Accuracy: 0.9922
Epoch: [ 0] [    3/  468], train_loss: 0.01935491, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [    4/  468], train_loss: 0.01449488, train_accuracy: 1.0000, test_Accuracy: 0.9688
Epoch: [ 0] [    5/  468], train_loss: 0.01817403, train_accuracy: 1.0000, test_Accuracy: 0.9922
Epoch: [ 0] [    6/  468], train_loss: 0.00883362, train_accuracy: 1.0000, test_Accuracy: 0.9922
Epoch: [ 0] [    7/  468], train_loss: 0.00758861, train_accuracy: 1.0000, test_Accuracy: 0.9688
Epoch: [ 0] [    8/  468], train_loss: 0.01288038, train_accuracy: 1.0000, test_Accuracy: 0.9844
Epoch: [ 0] [    9/  468], train_loss: 0.03034921, train_accuracy: 0.9922, test_Accuracy: 0.9766
Epoch: [ 0] [   10/  468], tra

Epoch: [ 0] [   85/  468], train_loss: 0.02376423, train_accuracy: 0.9844, test_Accuracy: 0.9766
Epoch: [ 0] [   86/  468], train_loss: 0.02107294, train_accuracy: 0.9922, test_Accuracy: 0.9766
Epoch: [ 0] [   87/  468], train_loss: 0.02653600, train_accuracy: 1.0000, test_Accuracy: 0.9766
Epoch: [ 0] [   88/  468], train_loss: 0.01697588, train_accuracy: 1.0000, test_Accuracy: 0.9688
Epoch: [ 0] [   89/  468], train_loss: 0.01749391, train_accuracy: 1.0000, test_Accuracy: 0.9766
Epoch: [ 0] [   90/  468], train_loss: 0.02105181, train_accuracy: 0.9922, test_Accuracy: 0.9609
Epoch: [ 0] [   91/  468], train_loss: 0.03504552, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [   92/  468], train_loss: 0.04243726, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [   93/  468], train_loss: 0.04755305, train_accuracy: 0.9766, test_Accuracy: 0.9766
Epoch: [ 0] [   94/  468], train_loss: 0.01740980, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [   95/  468], tra

Epoch: [ 0] [  170/  468], train_loss: 0.02452173, train_accuracy: 0.9922, test_Accuracy: 0.9922
Epoch: [ 0] [  171/  468], train_loss: 0.01808504, train_accuracy: 1.0000, test_Accuracy: 0.9844
Epoch: [ 0] [  172/  468], train_loss: 0.00661217, train_accuracy: 1.0000, test_Accuracy: 0.9766
Epoch: [ 0] [  173/  468], train_loss: 0.00722342, train_accuracy: 1.0000, test_Accuracy: 0.9766
Epoch: [ 0] [  174/  468], train_loss: 0.01604906, train_accuracy: 0.9922, test_Accuracy: 0.9609
Epoch: [ 0] [  175/  468], train_loss: 0.00755601, train_accuracy: 1.0000, test_Accuracy: 0.9844
Epoch: [ 0] [  176/  468], train_loss: 0.03857094, train_accuracy: 0.9922, test_Accuracy: 0.9688
Epoch: [ 0] [  177/  468], train_loss: 0.01453690, train_accuracy: 1.0000, test_Accuracy: 0.9688
Epoch: [ 0] [  178/  468], train_loss: 0.01907469, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  179/  468], train_loss: 0.02752090, train_accuracy: 0.9922, test_Accuracy: 0.9922
Epoch: [ 0] [  180/  468], tra

Epoch: [ 0] [  255/  468], train_loss: 0.00535039, train_accuracy: 1.0000, test_Accuracy: 0.9766
Epoch: [ 0] [  256/  468], train_loss: 0.05441321, train_accuracy: 0.9688, test_Accuracy: 0.9609
Epoch: [ 0] [  257/  468], train_loss: 0.02352593, train_accuracy: 1.0000, test_Accuracy: 0.9844
Epoch: [ 0] [  258/  468], train_loss: 0.00896364, train_accuracy: 1.0000, test_Accuracy: 0.9688
Epoch: [ 0] [  259/  468], train_loss: 0.00746192, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  260/  468], train_loss: 0.00562308, train_accuracy: 1.0000, test_Accuracy: 1.0000
Epoch: [ 0] [  261/  468], train_loss: 0.02505541, train_accuracy: 0.9922, test_Accuracy: 0.9766
Epoch: [ 0] [  262/  468], train_loss: 0.01801251, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [  263/  468], train_loss: 0.01506346, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [  264/  468], train_loss: 0.01549227, train_accuracy: 1.0000, test_Accuracy: 0.9688
Epoch: [ 0] [  265/  468], tra

Epoch: [ 0] [  342/  468], train_loss: 0.01162244, train_accuracy: 1.0000, test_Accuracy: 0.9922
Epoch: [ 0] [  343/  468], train_loss: 0.00906869, train_accuracy: 1.0000, test_Accuracy: 0.9453
Epoch: [ 0] [  344/  468], train_loss: 0.02852953, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [  345/  468], train_loss: 0.01088741, train_accuracy: 1.0000, test_Accuracy: 0.9922
Epoch: [ 0] [  346/  468], train_loss: 0.00944397, train_accuracy: 1.0000, test_Accuracy: 0.9531
Epoch: [ 0] [  347/  468], train_loss: 0.03990732, train_accuracy: 0.9844, test_Accuracy: 0.9844
Epoch: [ 0] [  348/  468], train_loss: 0.03656855, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [  349/  468], train_loss: 0.03537538, train_accuracy: 0.9844, test_Accuracy: 0.9766
Epoch: [ 0] [  350/  468], train_loss: 0.02319093, train_accuracy: 0.9922, test_Accuracy: 0.9844
Epoch: [ 0] [  351/  468], train_loss: 0.02209580, train_accuracy: 0.9922, test_Accuracy: 0.9922
Epoch: [ 0] [  352/  468], tra

Epoch: [ 0] [  427/  468], train_loss: 0.00965522, train_accuracy: 1.0000, test_Accuracy: 0.9922
Epoch: [ 0] [  428/  468], train_loss: 0.01772698, train_accuracy: 1.0000, test_Accuracy: 0.9609
Epoch: [ 0] [  429/  468], train_loss: 0.02488721, train_accuracy: 0.9844, test_Accuracy: 0.9688
Epoch: [ 0] [  430/  468], train_loss: 0.01087871, train_accuracy: 1.0000, test_Accuracy: 0.9922
Epoch: [ 0] [  431/  468], train_loss: 0.01687258, train_accuracy: 1.0000, test_Accuracy: 0.9609
Epoch: [ 0] [  432/  468], train_loss: 0.00577622, train_accuracy: 1.0000, test_Accuracy: 0.9688
Epoch: [ 0] [  433/  468], train_loss: 0.05257224, train_accuracy: 0.9844, test_Accuracy: 0.9766
Epoch: [ 0] [  434/  468], train_loss: 0.01971159, train_accuracy: 1.0000, test_Accuracy: 0.9688
Epoch: [ 0] [  435/  468], train_loss: 0.00840615, train_accuracy: 1.0000, test_Accuracy: 0.9844
Epoch: [ 0] [  436/  468], train_loss: 0.00467954, train_accuracy: 1.0000, test_Accuracy: 0.9844
Epoch: [ 0] [  437/  468], tra